# Frontier Models

This section uses AWS Bedrock to run inference tests using the Claude and Open AI models. 

In [27]:
!pip -q install langchain_aws langchain

In [2]:
%package install boto3=1.34.51 instructor

Running: micromamba install boto3=1.34.51 instructor --yes --quiet --log-level=error

Note: Packages not from Bloomberg channels are not vetted by Bloomberg.
Please restart the Jupyter kernel if you run into any issues after installing or updating packages via %package.



In [115]:
import json
import boto3
import botocore
import datetime
from tqdm import tqdm

from prompts import SYSTEM_PROMPTS
from IPython.display import Markdown, display


In [2]:
# boto3 must be version 1.34.51 or higher
boto3.__version__

'1.34.51'

## Initial Bedrock Test
This is an initial run of Bedrock with Anthropic to see how it responds to the financial analysis task

In [71]:
# Bedrock client initialise
config = botocore.config.Config(read_timeout=1000)
boto3_bedrock = boto3.client('bedrock-runtime',config=config)
# set the model
model = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'
# load prompts into memory
with open('Data/prompts.json', 'rb') as f:
    prompts = json.load(f)

In [4]:
# load the first prompt as a test. Use the chain of thought system prompt
prompt = f"{SYSTEM_PROMPTS['CoTDetailed_Claude']['prompt']} context: {prompts[0]['prompt'][1]['content']}"
prompt

"You are a financial analyst and must make a buy, sell, hold decision on a company based only on the provided financial statement. Your goal is to buy stocks you think will increase over the next financial period and sell stocks you think will decline. Think step-by-step through the financial statement analysis workflow. Your report should have the following sections: 1. Analysis of current profitability, liquidity, solvency and efficiency ratios; 2. time-series analysis across the ratios; 3. Analysis of financial performance; 4. Stock Price analysis; 5. Decision Analysis looking at the positive and negative factors as well as the weighting in the final decision; 6. Final Decision of BUY, SELL or HOLD and the reason for this. The final decision should have a confidence score. Always state the formulas for any calculations you will use. Your answer should have a decision of 'BUY', 'SELL' or 'HOLD' as well as a confidence score. If you are not sure of the answer then lower the confidence

In [5]:
def setup_claude_request(prompt: str) -> str:
    
    native_request = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 5000,
        "temperature": 0.7,
        "messages": [
            {
                "role":"user",
                "content": [{"type":"text", "text":prompt}]
            }
        ]
    }
    
    return json.dumps(native_request)

In [6]:
GUARDRAIL_ID = os.environ['BEDROCK_GUARDRAIL_ID']
GUARDRAIL_VERSION = os.environ['BEDROCK_GUARDRAIL_VERSION']

model_id = model


def invoke_model_claude(prompt: dict, model_id: str) -> str:
    """Invoke the model using Bedrock. This is specifically designed for Anthropic models"""
    accept = 'application/json'
    contentType = 'application/json'

    # set up the request
    request = setup_claude_request(prompt)
    try:
        
        response = boto3_bedrock.invoke_model(
            body=request, 
            modelId=model_id, 
            accept=accept, 
            contentType=contentType,
            trace = "ENABLED"
        )
        response_body = json.loads(response.get('body').read())
        return response_body.get('content')[0].get('text') #Anthropic response template
    
    except botocore.exceptions.ClientError as error:
        
        if error.response['Error']['Code'] == 'AccessDeniedException':
               print(f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubeshoot this issue please refer to the following resources.\
                     \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                     \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
            
        else:
            raise error

In [7]:
response = invoke_model_claude(prompt, model_id)

In [18]:
response

"# Financial Analysis Report\n\n## 1. Analysis of Current Profitability, Liquidity, Solvency, and Efficiency Ratios\n\n### Profitability Ratios:\n\n**Gross Profit Margin**\n- Formula: Gross Profit / Revenue\n- Current (t): $3,786,000,000 / $8,111,000,000 = 46.68%\n\n**Operating Profit Margin**\n- Formula: Operating Income / Revenue\n- Current (t): $1,325,000,000 / $8,111,000,000 = 16.34%\n\n**Net Profit Margin**\n- Formula: Net Income / Revenue\n- Current (t): $969,000,000 / $8,111,000,000 = 11.95%\n\n**Return on Assets (ROA)**\n- Formula: Net Income / Total Assets\n- Current (t): $969,000,000 / $44,659,000,000 = 2.17%\n\n**Return on Equity (ROE)**\n- Formula: Net Income / Total Equity\n- Current (t): $969,000,000 / $10,126,000,000 = 9.57%\n\n### Liquidity Ratios:\n\n**Current Ratio**\n- Formula: Current Assets / Current Liabilities\n- Current (t): $12,971,000,000 / $9,222,000,000 = 1.41\n\n**Quick Ratio**\n- Formula: (Current Assets - Inventory) / Current Liabilities\n- Current (t): (

## Langgraph version
This is the langgraph version in prep for multi-agentic AI system

In [73]:
# Set up and configure the model settings
model_llm = ChatBedrock(
    client = boto3_bedrock,
    model = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0',
    temperature = 0.7,
    max_tokens=None,
)

In [35]:
FA_ANALYSIS_PROMPT = SYSTEM_PROMPTS['CoTDetailed_OA']['prompt']

In [23]:
prompt = f"You are an assistant to a financial analyst. You are responsible for formatting the documents that the analyst produces into a machine readable format. You take the document and convert it into the structured output. Do not leave anything out of the analysts report and do not summarize. Do not add anything to the analysts report. Find the investment decision. Find the conclusion. Add all of the wording of the thought process into the steps section. context: {response}"
structured_llm = model_llm.with_structured_output(FinancialOutput)
formatted_response = structured_llm.invoke(prompt)

In [24]:
formatted_response

FinancialOutput(decision='SELL', reason='The company shows significant deterioration in profitability metrics with operating margin dropping from 25.2% to 16.3% and net profit margin falling from 19.8% to 11.9%. This profitability decline is accompanied by a concerning increase in operating expenses (37.3% year-over-year) without corresponding revenue growth. The balance sheet shows increasing financial leverage with debt-to-equity ratio rising to 3.41 and a dramatic 70% reduction in cash and cash equivalents.\n\nThe stock price has already begun reflecting these fundamental weaknesses, declining from January to February 2020. Given the combination of deteriorating operational performance, weakening balance sheet, and negative stock price momentum, the outlook for the next financial period appears negative. The relatively stable revenue base prevents an even higher confidence in the sell recommendation, but the weight of negative factors strongly suggests selling the stock before furth

## Run Inference Loop for Dow Jones

In [109]:
from langchain_aws import ChatBedrock
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.rate_limiters import InMemoryRateLimiter
import concurrent.futures

In [110]:
# set up the LLM in Bedrock
rate_limiter = InMemoryRateLimiter(
    requests_per_second=5,
    check_every_n_seconds=1,
    max_bucket_size=10,
)

llm = ChatBedrock(
    client = boto3_bedrock,
    model = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0',
    temperature = 0.7,
    max_tokens=4000,
    rate_limiter = rate_limiter
)

In [96]:
prompt_template = PromptTemplate.from_template(SYSTEM_PROMPTS['CoTDetailed_Claude']['prompt'])

In [77]:
def run_claude(prompt: dict, llm: RunnableSequence) -> dict:
    prompt_in = prompt_template.format(financials=prompt['prompt'][1]['content'])
    output = llm.invoke(prompt_in)
    decision_dict = {
        'date': prompt['date'],
        'security': prompt['security'],
        'response': output
    }
    return decision_dict

In [85]:
%%time
# This creates the initial response - run this in parallel with 100 executors
data = []
with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    futures = [executor.submit(run_claude, prompt, llm) for prompt in prompts]
    data = [f.result() for f in futures]

CPU times: user 7.08 s, sys: 505 ms, total: 7.58 s
Wall time: 9min 56s


In [98]:
cleaned_output = [{'date': i['date'], 
             'security': i['security'], 
             'response': i['response'].content} 
            for i in data]

In [99]:
with open('Data/Results - Bedrock.json', 'w') as f:
    json.dump(cleaned_output, f)

## Convert to JSON format
This is needed to run the trade analytics and compare to other LLMs

In [111]:
# STEP 1: Set up the structured output
class FinancialOutput(BaseModel):
    """Answer to the users question along with justification"""
    decision: str = Field(..., description="The investment decision, either BUY, SELL or HOLD")
    reason: str = Field(..., description="final decision reason")
    confidence: int = Field(..., description="How confident you are of the decision")

In [112]:
# STEP 2: 
system_prompt = f"You are an assistant to a financial analyst. You are responsible for formatting the documents that the analyst produces into a machine readable format. You take the document and convert it into the structured output. Do not leave anything out of the analysts report and do not summarize. Do not add anything to the analysts report. Find the investment decision. Find the conclusion. Add all of the wording of the thought process into the steps section. context: {response}"

prompt_template = PromptTemplate.from_template(system_prompt)
structured_llm = llm.with_structured_output(FinancialOutput)
# formatted_response = structured_llm.invoke(prompt)

In [113]:
# STEP 3: Set up 
def run_cleanup(prompt, llm: RunnableSequence) -> dict:
    thought_process = prompt['response']
    prompt_in = prompt_template.format(response=thought_process)
    response = llm.invoke(prompt_in)
    final_dict = {
        'date': prompt['date'],
        'security': prompt['security'],
        'response': {
            'decision': response.decision,
            'reason': response.reason,
            'confidence': response.confidence,
            'thought_process': thought_process
        }
    }
    return final_dict

In [114]:
%%time
responses = []
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(run_cleanup, prompt, structured_llm) for prompt in cleaned_output]
    responses = [f.result() for f in futures]

CPU times: user 5.64 s, sys: 107 ms, total: 5.75 s
Wall time: 7min 32s


In [116]:
# STEP 5: convert to the correct format
final_json = {
    'run_date': str(datetime.datetime.now()),
    'system_prompt': SYSTEM_PROMPTS['CoTDetailed_Claude']['prompt'],
    'dataset': 'data_quarterly_pit_indu',
    'model': model_id,
    'results': responses
}

In [119]:
with open(f'Results/results - {str(datetime.datetime.now())}.json', 'w') as f:
    json.dump(final_json, f)